# **Übung 8, Teil 2: Numerische Lösung der nicht-viskosen Burgersgleichung**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Erhöhen der Plot-Auflösung
plt.rcParams["figure.dpi"] = 140

XLIM = [0, 1]
YLIM = [-0.5, 1.5]

### **8.2.1 Numerische Parameter**

In [ ]:
m = 100             # Anzahl der Stützstellen
dx = 1 / m          # räumliche Schrittweite
n = 250              # Anzahl Zeitschritte
cfl = 1             # CFL-Zahl
k2 = 0.5            # Skalierungsterm

In [ ]:
def initial_cond(x):
    """ Erzeugt eine Testfunktion mit Diskontinuität """
    u_0 = np.ones_like(x) 
    u_0[x > 0.2] = 0    
    return u_0


def prop_speed(u, i, j):
    """ Berechnet die analytische Stoßgeschwindigkeit """
    return 0.5 * (u[i-1, j] + u[i+1, j])


def get_dt(u , j):
    """ Berechnet die neue Zeitschrittweite dt basierend auf der maximalen Ausbreitungsgeschwindigkeit des letzten Zeitschritts """
    u_max = np.max(u[:, j])
    u_min = np.min(u[:, j])

    return dx * cfl / max(u_max, abs(u_min))


def apply_BC(u, j):
    """ Setzt die Randbedinungen """
    u[0, j] = 1
    u[-1, j] = 0        # theoretisch überflüssig, da u als Matrix mit Werten 0 initialisiert wird

    c0 = prop_speed(u, 1, j-1)
    if c0 < 0:
        u[0, j] = u[1, j-1]
    
    c1 = prop_speed(u, m-1, j-1)
    if c1 > 0:
        u[-1, j] = u[-2, j - 1]

    return u


def d2(u, i, j): 
    """ 2. Differenzen """
    return (abs(0.5 * (u[i+1, j] + u[i, j])) * (u[i+1, j] - u[i, j]) - abs(0.5 * (u[i-1, j] + u[i, j])) * (u[i, j] - u[i-1, j])) / dx


# Löser-Schleife
def simple_solver(u, dx, n, m):
    """ 
    Zeitlich:   Einstufiges, explizites Zeitschrittverfahren (1. Ordnung) 
    Räumlich:   Zentrales Schema + 2. Differenzen (1. Ordnung)
    """
    for j in range(1, n + 1):
        u = apply_BC(u, j)
        dt = get_dt(u, j-1)
        for i in range(1, m):
            c = prop_speed(u, i, j)
            u[i, j] = u[i, j-1] - dt * (c * (u[i+1, j-1] - u[i-1, j-1]) / 2 / dx - k2 * d2(u, i, j-1))
            
    return u

### **8.2.2 Gitterpunkte und Anfangslösung erzeugen**

In [ ]:
# Gitterpunkte 
x = np.linspace(0, 1, m + 1)   

# Lösungsmatrix für u initialisieren:
#   - die räumliche Ausbreitung zum Zeitpunkt t ist in Spalte t untergebracht u[:, t]
#   - die zeitliche Ausbreitung am Ort x ist in Reihe x untergebracht u[x, :]
u = np.zeros((m + 1, n + 1))   

# Anfangsbedingung
u[:, 0] = initial_cond(x)

In [ ]:
# Visualisierung der Anfangslösung
plt.figure(figsize=(6, 3))
plt.plot(x, initial_cond(x), color="gray")
plt.title('Diskontinuität (Stoß) als Anfangslösung')
plt.xlabel('x')
plt.ylabel(r'$u(x, t=0)$')
plt.grid(True)
plt.xlim(XLIM)
plt.ylim(YLIM)
plt.tight_layout()
plt.show()

### **8.2.3 Numerische Lösung**

In [ ]:
# Animation der Lösung
def animation_u(title, YLIM, u=u):
    fig, ax = plt.subplots(figsize=(6, 3))
    ax.set_xlim(XLIM)
    ax.set_ylim(YLIM)
    ax.set_title(f"{title}, t = 0")
    ax.plot(x, u[:, 0], lw=2, color="gray", label='Anfangslösung')
    line, = ax.plot([], [], lw=2, color="firebrick", label='Lösung')
    ax.set_xlabel('x')
    ax.set_ylabel(r'$u(x, t)$')
    ax.grid(True)
    ax.legend(loc="lower center")
    plt.tight_layout()

    def animate(j):
        line.set_data(x, u[:, j])
        ax.set_title(f"{title}, t = {j*get_dt(u,j):.2f} s")
        return line,

    ani = animation.FuncAnimation(fig, animate, frames=n, interval=25, blit=True)
    plt.close(fig)
    return ani

In [ ]:
u = simple_solver(u, dx, n, m)

ani = animation_u("Lösung der Wellengleichung", YLIM=YLIM)
HTML(ani.to_jshtml())